---
title: "K-means + ObservableJS"
author: Olivier Malahel
format: html
---

## import des bibliothèques Observable
```{ojs}
import { Inputs } from "@observablehq/stdlib"
import { Plot } from "@observablehq/plot"
```

## Génération des notes dans quatre matières

```{js}
// 1. Installer les paquets (npm) ou charger via CDN avant d’exécuter ce code :
//    npm install seedrandom d3-random

import seedrandom from "seedrandom";
import { randomNormal } from "d3-random";

// 2. Paramètres
const seed       = "123";
const n_students = 30;

// 3. Initialisation du générateur seedé
const rng    = seedrandom(seed);
const normal = randomNormal.source(rng);

// 4. Génération du tableau “notes” avec colonne Etudiant
const notes = Array.from({ length: n_students }, (_, i) => ({
  Etudiant:       `E${String(i + 1).padStart(2, "0")}`,
  Mathematiques:  +normal(14,   2  ).toFixed(1),
  Physique:       +normal(12,   3  ).toFixed(1),
  Informatique:   +normal(15,   2.4).toFixed(1),
  Chimie:         +normal(11,   3  ).toFixed(1)
}));

// 5. Simuler rownames(notes) et notes$Etudiant <- NULL
//    On crée un objet dont chaque clef est l’étudiant et la valeur l’objet de ses notes
const notesByStudent = Object.fromEntries(
  notes.map(({ Etudiant, ...scores }) => [Etudiant, scores])
);

// 6. Affichage final
console.log(notesByStudent);
```

## Affichage du tableau des notes

```{ojs}
viewof data = Inputs.table(initial, {
  columns: ["Etudiant", "Maths", "Physique", "Info", "Chimie"],
  label: "Éditez les notes des étudiants"
})
```

### Curseur pour faire varier le nombre de clusters 

```{ojs}
// 3. Slider pour choisir k
viewof k = Inputs.range([1, 6], {
  value: 3,
  step: 1,
  label: "Nombre de clusters (k)"
})

// 4. Calcul du k-means
points = data.map(d => ({
  x: +d.Maths,      // abscisse = Mathématiques
  y: +d.Chimie,     // ordonnée = Chimie
  name: d.Etudiant
}))

function kmeans(points, k, maxIter = 50) {
  let centroids = points
    .slice()
    .sort(() => Math.random() - 0.5)
    .slice(0, k)
    .map(p => ({x: p.x, y: p.y}))
  let assignments = new Array(points.length)

  for (let iter = 0; iter < maxIter; iter++) {
    // Affectation
    for (let i = 0; i < points.length; i++) {
      let best = 0, bestDist = Infinity
      for (let j = 0; j < k; j++) {
        const dx = points[i].x - centroids[j].x
        const dy = points[i].y - centroids[j].y
        const d2 = dx*dx + dy*dy
        if (d2 < bestDist) { bestDist = d2; best = j }
      }
      assignments[i] = best
    }
    // Recalcul des centroïdes
    const sums = Array.from({length: k}, () => ({x:0,y:0,count:0}))
    for (let i = 0; i < points.length; i++) {
      const c = assignments[i]
      sums[c].x += points[i].x
      sums[c].y += points[i].y
      sums[c].count++
    }
    let moved = false
    for (let j = 0; j < k; j++) {
      if (!sums[j].count) continue
      const nx = sums[j].x / sums[j].count
      const ny = sums[j].y / sums[j].count
      if (nx !== centroids[j].x || ny !== centroids[j].y) moved = true
      centroids[j].x = nx; centroids[j].y = ny
    }
    if (!moved) break
  }

  return {centroids, assignments}
}

result = kmeans(points, k)
centroids = result.centroids
assignments = result.assignments
```

## Tableau des notes avec les clusters

```{ojs}
// 5. Tableau des clusters
dataWithCluster = data.map((d,i) => ({
  ...d,
  Cluster: `Cluster ${assignments[i]+1}`
}))
viewof clusterTable = Inputs.table(dataWithCluster, {
  columns: ["Etudiant","Maths","Physique","Info","Chimie","Cluster"],
  label: "Profils et clusters"
})

// 6. Tableau des centroïdes
centroidsWithLabel = centroids.map((c,i) => ({
  Cluster: `Cluster ${i+1}`,
  Centre_Maths: c.x,
  Centre_Chimie: c.y
}))
viewof centroidTable = Inputs.table(centroidsWithLabel, {
  columns: ["Cluster","Centre_Maths","Centre_Chimie"],
  label: "Centres des clusters"
})
```

## Représentation graphique 

Ici, les notes des élèves en mathématiques et en chimie, sont représentées dans le plan  

```{ojs}
// 7. Affichage du scatter : Maths vs Chimie
Plot.plot({
  width: 600,
  height: 400,
  x: {label: "Mathématiques"},
  y: {label: "Chimie"},
  marks: [
    // points
    Plot.dot(
      points.map((p,i) => ({...p, cluster: String(assignments[i])})),
      {x:"x", y:"y", fill:"cluster", title:"name"}
    ),
    // croix = centroïdes
    Plot.dot(
      centroids.map((c,i) => ({x:c.x, y:c.y, cluster:String(i)})),
      {x:"x", y:"y", stroke:"black", symbol:"cross", strokeWidth:2}
    )
  ]
})
```